# Create Heat Map for significant Cell Cycle genes

This notebook looks at the significant genes in at least one cancer in the DNA Replication pathway. Pancancer heat maps are created with circle size showing significance and color showing differences in median.

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import plot_utils as p

# Step 1: Run GSEA for significant genes in at least 1 cancer

First read in sig_pval_all_proteins.csv into a df. This csv file contains only genes with a significant p-value in at least one cancer. Then run GSEA using a list of genes from the df.

In [3]:
root = R'~\Github\WhenMutationsDontMatter\PTEN\Step_3_trans_effect\csv'
sig_df = pd.read_csv(root+R'\sig_pval_heatmap.csv')

prot_list = list(sig_df.Proteomics) # list of genes with a sig pval in >= 1 cancer
prot_enr = gp.enrichr(gene_list = prot_list, description='Tumor_partition', gene_sets='Reactome_2016', 
                       outdir='/Enrichr')

In [4]:
prot_enr.res2d.iloc[[9]]

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
9,Cell Cycle Homo sapiens R-HSA-1640170,160/566,1.870504e-22,2.861870e-20,0,0,2.165343,108.333543,MDC1;ANKLE2;NUP107;NUP188;ZWILCH;BUB1B;PPP2R2A...,Reactome_2016


# Step 2: Get the list of significant genes 

In [25]:
index = 9
trans = prot_enr.res2d.Genes[index]
genes = trans.split(';')
print(prot_enr.res2d.Term[index])
print('total genes:',len(genes))

Cell Cycle Homo sapiens R-HSA-1640170
total genes: 160


# Step 3: Create HeatMap

Slice out genes from the DNA Replication pathway from the df with genes sig in > 1 cancer.

In [26]:
# sig > 1 cancer
bool_df = sig_df.Proteomics.isin(genes)
plot_df = sig_df[bool_df]
len(plot_df.Proteomics.unique())

160

In [27]:
# Only include p-values < a certain cutoff
a = 0.01
plot_df = plot_df.loc[plot_df['P_Value'] <= a]

In [30]:
p.plotCircleHeatMap(plot_df, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width= 3500)

Loading BokehJS ...

# Step 4: Create a HeatMap with both +/- differences in median

 Read in the pos_neg_df.csv to create a df with only genes that have a pos and neg difference in median in different cancers. Slice out the genes that have a pos and neg difference in median in the pathway using the list of genes with a significant p-value in the pathway. 

In [8]:
pos_neg_df = pd.read_csv(root+R'\pos_neg_df.csv')

In [31]:
get = pos_neg_df.Proteomics.isin(genes) # bool df where True has both pos and neg
genes_pn = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn.Proteomics.unique()

array(['MAPK3', 'CENPM', 'LMNB1', 'MCM6', 'MCM4', 'GINS3', 'RFC5', 'MCM5',
       'MCM2', 'PCM1', 'MCM7', 'NUP37', 'MCM3', 'NUP153', 'SPC24', 'SMC4',
       'CDK11B', 'NSL1', 'SMC2', 'SPC25', 'NCAPD2', 'RFC2', 'NUF2',
       'GTSE1', 'RFC4', 'TOPBP1', 'NCAPG', 'CHEK1', 'CENPK', 'CENPF',
       'GINS4', 'NDC80', 'POLA2', 'PPP2R2A', 'PPP1R12B', 'CDCA5', 'HDAC1',
       'TPX2', 'NDEL1', 'MIS18A', 'GINS2', 'BUB1B', 'TOP2A', 'E2F5',
       'RCC2', 'MDC1', 'DKC1', 'NUP210', 'TOP3A', 'NEK9', 'PLK1', 'ODF2',
       'CEP78', 'GORASP1', 'SYNE1', 'PSMB8', 'TP53', 'PSMB9'],
      dtype=object)

In [32]:
# Only include p-values < a certain cutoff
genes_pn = genes_pn.loc[genes_pn['P_Value'] <= a]

In [34]:
p.plotCircleHeatMap(genes_pn, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=1000)

Loading BokehJS ...

# Sig in multiple cancers

In [12]:
mult_sig_df = pd.read_csv(root+R'\mult_sig_pval_heatmap.csv')

mult_sig_list = list(mult_sig_df.Proteomics.unique()) # list of genes with a sig pval in > 1 cancer
enr2 = gp.enrichr(gene_list = mult_sig_list, description='Tumor_partition', gene_sets='Reactome_2016', 
                       outdir='/Enrichr')

In [13]:
enr2.res2d.head(3)

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Gene Expression Homo sapiens R-HSA-74160,74/1631,2.081067e-17,3.184033e-14,0,0,2.917745,112.073703,WDR3;WDR4;PTEN;HNRNPU;NAT10;ZC3H8;ADAR;PPP2R2A...,Reactome_2016
1,"Cell Cycle, Mitotic Homo sapiens R-HSA-69278",37/462,2.789117e-16,2.133675e-13,0,0,5.150262,184.459924,TOP2A;AHCTF1;PCNA;MCM7;DCTN1;BUB1B;PPP2R2A;ORC...,Reactome_2016
2,Cell Cycle Homo sapiens R-HSA-1640170,38/566,3.204908e-14,1.634503e-11,0,0,4.317544,134.152603,TOP2A;AHCTF1;PCNA;MCM7;DCTN1;BUB1B;PPP2R2A;ORC...,Reactome_2016
3,Processing of Capped Intron-Containing Pre-mRN...,23/193,3.858260e-14,1.475784e-11,0,0,7.663729,236.701740,RANBP2;DDX5;NUP210;CPSF1;NUP155;NCBP1;NUP133;D...,Reactome_2016
4,DNA strand elongation Homo sapiens R-HSA-69190,11/32,1.039399e-12,3.180562e-10,0,0,22.106109,609.960125,GINS2;RFC3;PCNA;RFC4;MCM7;RFC2;GINS4;MCM3;MCM4...,Reactome_2016


In [59]:
index2 = 2
trans2 = enr2.res2d.Genes[index2]
genes_mult = trans2.split(';')
print(enr2.res2d.Term[index2])
print('total genes:',len(genes_mult))

Cell Cycle Homo sapiens R-HSA-1640170
total genes: 38


In [60]:
bool_df = mult_sig_df.Proteomics.isin(genes_mult)
plot_df2 = mult_sig_df[bool_df]
plot_df2.Proteomics.unique()

array(['DCTN1', 'ACTR1A', 'MCM4', 'MAD2L1', 'PCNA', 'MCM5', 'MCM2',
       'PCM1', 'MCM7', 'TPR', 'MCM3', 'RFC3', 'CDK11B', 'XPO1', 'PRKCB',
       'RFC2', 'NUF2', 'RAB8A', 'RFC4', 'TOPBP1', 'RANBP2', 'CENPF',
       'NUP155', 'GINS4', 'NDC80', 'OPTN', 'ORC5', 'PPP2R2A', 'PPP2R2D',
       'TPX2', 'NUP85', 'RAE1', 'GINS2', 'BUB1B', 'TOP2A', 'NUP133',
       'AHCTF1', 'NUP210'], dtype=object)

In [61]:
# Only include p-values < a certain cutoff
a = 0.05
plot_df2 = plot_df2.loc[plot_df2['P_Value'] <= a]

In [62]:
p.plotCircleHeatMap(plot_df2, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=1000)

Loading BokehJS ...

In [49]:
get = pos_neg_df.Proteomics.isin(genes_mult) # bool df where True has both pos and neg
genes_pn_mult = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn_mult.Proteomics.unique()

array(['MCM4', 'MCM5', 'MCM2', 'PCM1', 'MCM7', 'MCM3', 'CDK11B', 'RFC2',
       'NUF2', 'RFC4', 'TOPBP1', 'CENPF', 'GINS4', 'NDC80', 'PPP2R2A',
       'TPX2', 'GINS2', 'BUB1B', 'TOP2A', 'NUP210'], dtype=object)

In [50]:
# Only include p-values < a certain cutoff
genes_pn_mult = genes_pn_mult.loc[genes_pn_mult['P_Value'] <= a]

In [52]:
p.plotCircleHeatMap(genes_pn_mult, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=600)

Loading BokehJS ...

# Check other pathways the genes (+/- and sig in mult. cancers) are part of

In [54]:
#gnm = list(genes_pn_mult.Proteomics.unique())
#print(gnm)
enr3 = gp.enrichr(gene_list = genes_mult, description='Tumor_partition', gene_sets='Reactome_2016', 
                       outdir='/Enrichr')

In [56]:
enr3.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,"Cell Cycle, Mitotic Homo sapiens R-HSA-69278",37/462,2.464151e-60,3.770152e-57,0,0,42.150832,5785.338972,TOP2A;AHCTF1;PCNA;MCM7;DCTN1;BUB1B;PPP2R2A;ORC...,Reactome_2016
1,Cell Cycle Homo sapiens R-HSA-1640170,38/566,4.241800e-60,3.244977e-57,0,0,35.335689,4830.746211,TOP2A;AHCTF1;PCNA;MCM7;DCTN1;BUB1B;PPP2R2A;ORC...,Reactome_2016
2,DNA strand elongation Homo sapiens R-HSA-69190,11/32,2.952657e-23,1.505855e-20,0,0,180.921053,9385.596522,GINS2;RFC3;PCNA;RFC4;MCM7;RFC2;GINS4;MCM3;MCM4...,Reactome_2016
3,M Phase Homo sapiens R-HSA-68886,17/268,1.942762e-22,7.431065e-20,0,0,33.385703,1669.043485,RANBP2;AHCTF1;NUP210;NUP155;NUP133;PRKCB;BUB1B...,Reactome_2016
4,Synthesis of DNA Homo sapiens R-HSA-69239,12/97,2.042530e-19,6.250142e-17,0,0,65.111232,2802.057139,ORC5;GINS2;RFC3;RFC4;PCNA;MCM7;RFC2;GINS4;MCM3...,Reactome_2016


In [55]:
j = 2
print(enr3.res2d.Term[j])
enr3.res2d.Genes[j]

DNA strand elongation Homo sapiens R-HSA-69190


'GINS2;RFC3;PCNA;RFC4;MCM7;RFC2;GINS4;MCM3;MCM4;MCM5;MCM2'

In [23]:
# Look at p-values 
certain_gene = 'CHEK1'
all_df = pd.read_csv(root+R'\all_heatmap.csv')
gene_df = all_df.loc[all_df['Proteomics'] == certain_gene]
gene_df

,Proteomics,P_Value,Medians,Cancer
921,CHEK1,0.001264,0.461686,Gbm
11793,CHEK1,0.005425,0.312344,Hnscc
32436,CHEK1,0.923865,0.010050,Luad
35948,CHEK1,0.267259,0.231300,Lscc
46874,CHEK1,0.137476,0.536650,Brca
59014,CHEK1,0.404643,0.078545,Ov
66440,CHEK1,0.013974,-0.528000,En
